In [6]:
import pandas as pd
from dotenv import load_dotenv
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
load_dotenv()

True

In [8]:
HF_TOKEN = os.environ["HF_TOKEN"]

In [10]:
model_name = "facebook/opt-350m"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)

In [37]:
df = pd.read_csv("../../../data/europeancities-wikivoyage-tripadvisor/data-sources/wikivoyage/q_a/cities_qa_merged.csv")
print(df.shape)
df.head()

(5285, 3)


,city,prompt,answer
0,Belgrade,What is Belgrade known for?,"Belgrade is known for its vibrant nightlife, h..."
1,Belgrade,What is the history of Belgrade?,Belgrade has a long history dating back to the...
2,Belgrade,What is the climate like in Belgrade?,Belgrade has a temperate continental climate w...
3,Belgrade,What are the people like in Belgrade?,Belgradians are known for their friendliness a...
4,Belgrade,How can I get to Belgrade by plane?,Belgrade Nikola Tesla Airport (BEG) is the mai...


In [29]:
def formatting_prompts_func(example):
    output_texts = []
    max_length = 0
    idx = ""
    for i, row in example.iterrows():
        text = f"### Question: {example['prompt'][i]} ### Answer: {example['answer'][i]}"
        result = tokenizer(text)
        if len(result.input_ids) > max_length:
            max_length = len(result.input_ids)
            idx = example['prompt'][i]
        output_texts.append(text)
    return idx, max_length


In [30]:
formatting_prompts_func(df)

('What are some things to see in Sarajevo?', 1011)

In [33]:
df.loc[df['prompt']=='What are some things to see in Sarajevo?']["answer"].values

array(["Some of the top attractions in Sarajevo include:\n\n* Baščaršija: The historic district with cobbled streets, mosques, and oriental-style shops.\n* Sebilj: A wooden fountain in the middle of Pigeon Square.\n* Sacred Heart Cathedral: The largest Christian cathedral in Bosnia.\n* Morića Inn: The only preserved Ottoman Inn in Sarajevo.\n* Clock Tower: The tallest clock tower in Bosnia, dating back to the 17th century.\n* City Hall: An iconic pseudo-Moorish revival style building.\n* Avaz Twist Tower: A distinctive 176 m twisted tower.\n* Tunnel of Hope: A museum built over the tunnel that served as a lifeline to the besieged city during the war.\n* War Childhood Museum: A museum dedicated to the experience of childhood during the Siege of Sarajevo.\n* Latin Bridge: The bridge where Archduke Franz Ferdinand was assassinated.\n* Šeher-Čehajina Bridge: A stone bridge built in the 16th century.\n* Roman Bridge: A stone bridge crossing the Bosna River.\n* Goat's Bridge: An architectura

In [34]:
df.loc[df['prompt']=='What are some things to see in Sarajevo?']["answer"]

1562    Some of the top attractions in Sarajevo includ...
Name: answer, dtype: object

In [35]:
len(df.loc[df['prompt']=='What are some things to see in Sarajevo?']["answer"].values[0])

3887

In [36]:
# Assuming df is your DataFrame
filtered_df = df[df['answer'].apply(len) < 1000]
filtered_df.shape

(5246, 3)

In [38]:
df[df['answer'].apply(len) > 1000]

,city,prompt,answer
23,Belgrade,What is the history of Belgrade?,Belgrade has a long history dating back to the...
33,Belgrade,What are the public transportation options in ...,Belgrade has an extensive public transport net...
35,Belgrade,What are the taxi options in Belgrade?,"Taxis are cheap by European standards, though ..."
36,Belgrade,What are some of the things to see in Belgrade?,Belgrade city core is not too big. Everything ...
38,Belgrade,What are some places to eat in Belgrade?,Belgrade has hundreds of restaurants specializ...
48,Belgrade,How safe is Belgrade?,"Overall, Belgrade is a pretty safe city, but l..."
52,Belgrade,What are some health tips for visiting Belgrade?,"Belgrade's climate is generally temperate, so ..."
186,Budapest,What is the history of Budapest?,The history of Budapest dates back to the Celt...
569,Donetsk,What are some things to see in Donetsk?,"Donetsk offers a variety of attractions, inclu..."
1562,Sarajevo,What are some things to see in Sarajevo?,Some of the top attractions in Sarajevo includ...


In [39]:
formatting_prompts_func(filtered_df)

('What are some good places to stay in Salzburg?', 309)

In [41]:
filtered_df.to_csv("../../../data/europeancities-wikivoyage-tripadvisor/data-sources/wikivoyage/q_a/short_data.csv", index=False)